In [1]:
"""
Tutorial showing how to use Krylov's method to compute the coefficients of the 
characteristic polynomial of a matrix. 
(See Example 7.11.3 of Carl Meyer)
"""

import numpy as np
n = 5
A = np.random.randn(n,n)
b = np.random.randn(n,1)

# Initialize Krylov's sequence with just the vector b -- see p.646
Ks = [b]


In [13]:
def in_subspace(X, x):
    r1 = np.linalg.matrix_rank(X)
    r2 = np.linalg.matrix_rank(np.concatenate((X, x), axis=1))
    
    return r1 == r2
    
Ak = A
Ks = [b]
terminate = False

"""
Keep adding to the Krylov sequence until the vector that we add is a linear
combination of the previous vectors in the Krylov sequence.

This is necessary in order to construct the annihilating polynomial v()
for b relative to A as explained in p.646
"""
for k in range(1,n+1):
    Akb = Ak@b
    terminate = in_subspace(np.concatenate(Ks, axis=1),Akb)
    Ks.append(Akb)
    if terminate:
        break
    Ak = Ak@A

print(len(Ks))

6


In [22]:
"""
Find alpha coefficients of annihilating polynomial v()
by solving the linear system defined in p.646 for alpha
"""

Kj = np.concatenate(Ks[:-1], axis=1)
alpha = np.linalg.solve(Kj, Ks[-1])
print(alpha)

[[ 9.98017959]
 [ 3.45176124]
 [ 8.30062378]
 [-0.41550472]
 [-0.63241821]]


In [21]:
"""
The alpha coefficients above are the coefficients that we seek!
Let's check that we got the solution right
"""

import numpy as np

def characteristic_polynomial(M: np.ndarray) -> np.polynomial.polynomial.Polynomial:
    return np.polynomial.Polynomial.fromroots(np.linalg.eigvals(M))

characteristic_polynomial(A)

Polynomial([-9.98017959-2.72742800e-16j, -3.45176124+1.81157976e-16j,
       -8.30062378-1.29035375e-15j,  0.41550472-4.44089210e-16j,
        0.63241821+0.00000000e+00j,  1.        +0.00000000e+00j], domain=[-1.,  1.], window=[-1.,  1.])

In [23]:
"""
Our matrix was diagonalizable, that's why we already have the characteristic polynomial.
But if it weren't diagonalizable, we would need to proceed further by following
equations 7.11.7 and 7.11.8. This would require us to build the companion matrix etc. 
as below
"""


"""
Function to construct the companion matrix C as in 7.11.6
"""
def construct_companion_matrix(alpha):
    n = len(alpha)
    Cbl = np.eye(n-1)
    Cl = np.concatenate((np.zeros((1,n-1)), Cbl), axis=0)
    return np.concatenate((Cl, alpha), axis=1)


In [17]:
"""
Construct the companion matrix for our specific alphas.
"""

C = construct_companion_matrix(alpha)
print(C)

[[ 0.          0.          0.          0.          9.98017959]
 [ 1.          0.          0.          0.          3.45176124]
 [ 0.          1.          0.          0.          8.30062378]
 [ 0.          0.          1.          0.         -0.41550472]
 [ 0.          0.          0.          1.         -0.63241821]]


In [24]:
"""
According to Krylov's method (eq. 7.11.7), the matrix below ...
"""

Kj@C

array([[-2.73427318e+00,  4.00429951e+00, -5.87274058e+00,
        -3.02421990e+01,  3.83192646e+01],
       [ 3.75018363e-03, -2.75952197e+00, -2.75747994e+00,
         6.28349867e+00, -4.47116598e+01],
       [ 6.36030697e+00, -3.83399668e+00, -8.10912452e+00,
         4.33909424e+01, -5.81405288e+01],
       [-9.98739105e-02, -7.00352542e+00,  1.79886202e+01,
        -2.11216519e+01, -4.22729095e+01],
       [-1.09086426e+00,  9.78158997e+00,  3.30416497e+00,
         5.83166528e+00,  7.18333666e+01]])

In [25]:
"""
is equal to this one
"""
A@Kj

array([[-2.73427318e+00,  4.00429951e+00, -5.87274058e+00,
        -3.02421990e+01,  3.83192646e+01],
       [ 3.75018363e-03, -2.75952197e+00, -2.75747994e+00,
         6.28349867e+00, -4.47116598e+01],
       [ 6.36030697e+00, -3.83399668e+00, -8.10912452e+00,
         4.33909424e+01, -5.81405288e+01],
       [-9.98739105e-02, -7.00352542e+00,  1.79886202e+01,
        -2.11216519e+01, -4.22729095e+01],
       [-1.09086426e+00,  9.78158997e+00,  3.30416497e+00,
         5.83166528e+00,  7.18333666e+01]])

In [28]:
"""
And the moral of the story is that most information about A is
stored in the companion matrix C, which is upper Hessenberg,
and therefore can be used for efficient computations

(see p.650)
"""

'\nAnd the moral of the story is that most information about A is\nstored in the companion matrix C, which is upper Hessenberg,\nand therefore can be used for efficient computations\n\n(see p.650)\n'